Practical Exam Python


Part 2. Practical Question

This dataset contains HR Data Analytics. By using features, predict whether employee will leave company or not.
(“left” is target column).
You are asked to build model for predicting patients having heart disease.
1. Import data (HR_data.csv) and get familiarized with it. (show statistical summary, percentage of missing
values in each column and check duplicate values.) (6 points)
2. Show distribution of variables, correlation between them and check imbalance problem. (6 points)
3. Prepare data for modelling. (7 points)
4. Fit data into different models – Logistic regression, Decision Tree, Naive Bayes and XGBoost models. (10
points)
5. Print detailed reports regarding accuracy metrics for each model. (8 points)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [ ]:
df = pd.read_csv('/content/HR_data - HR_data - HR_data - HR_data - HR_data - HR_data - HR_data - HR_data.csv')
df

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.nunique()

In [ ]:
df.info()

In [ ]:
df['Age']

In [ ]:
df.isnull().mean() * 100

In [ ]:
df.hist(figsize=(10,10))
plt.show()

In [ ]:
df["Age"] = df["Age"].fillna(df["Age"].median())


In [ ]:
df.info()



In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(df.select_dtypes(include = 'number').corr(),annot=True)
plt.show()

In [ ]:
sns.countplot(x = 'left', data = df)
plt.title('Distribution of Left')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split


df_encoded = pd.get_dummies(df, columns=['salary', 'Department'])

X = df_encoded.drop('left', axis=1)
y = df_encoded['left']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Logistic regression, Decision Tree, Naive Bayes and XGBoost models.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'XGBoost': XGBClassifier()
}

fit_models = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    fit_models[name] = model

    y_pred = model.predict(X_test)

    print(f'\nModel: {name}')
    print('-------------------------')

    score = accuracy_score(y_test, y_pred)
    print(f"Accuracy Score: {score:.4f}")

    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


PART 2: Practical question

This is a dataset from the U.S. Census Bureau hosted by the Federal Reserve Economic Database (FRED).
The dataset consists of change in the number of population of X state from 01-01-1952 till 12-01-2019.
You are asked to build a forecasting model using Deep Learning techniques.
1. Import data (pop.csv) and get familiarized with it. (show statistics summary, search for missing values etc.)
(5 points)
2. Visualize ‘value’ column according to time. Find correlation between them. (3 points)
3. Prepare data for modelling. (5 points)
4. Define model parameters. Fit data into Prophet model. (7 points)
5. Plot components of Prophet model, visualize predicted and actual values. (3 points)

In [ ]:
df1 = pd.read_csv('/content/POP - POP - POP - POP - POP - POP - POP - POP.csv')
df1

In [ ]:
df1.info()

In [ ]:
df1.describe()

In [ ]:
df1.isna().sum()*100

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])


fig = px.line(
    df1,
    x = 'date',
    y = 'value',
    title = 'Closing Price over line',
    markers = True
)

fig.update_layout(
    template = 'plotly_dark',
    xaxis_title = 'Date',
    yaxis_title = 'Clothing prise (USD)',
    hovermode = 'x unified'
)

fig.show()

In [ ]:
pip install bayesian-optimization


In [ ]:
import logging
import pandas as pd
import numpy as np
from prophet import Prophet
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error


In [ ]:
df1 = df1[['date','value']]
df1.columns = ['ds', 'y']
df1['ds'] = pd.to_datetime(df1['ds'])

In [ ]:
df1

In [ ]:
train_size = int(0.97 * len(df1))
train_df = df1.iloc[:train_size]
test_df = df1.iloc[train_size:]
print(len(test_df))
test_df

In [ ]:
def Prophet_cv(changepoint_prior_scale, seasonality_prior_scale, holidays_prior_scale):
  model = Prophet(
      changepoint_prior_scale = changepoint_prior_scale,
      seasonality_prior_scale = seasonality_prior_scale,
      holidays_prior_scale = holidays_prior_scale
  )
  model.fit(train_df)
  future = model.make_future_dataframe(periods = len(test_df))
  forecast = model.predict(future)
  y_pred = forecast['yhat'].iloc[-len(test_df):].values
  y_true = test_df['y'].values
  mae = mean_absolute_error(y_true, y_pred)
  return -mae

pbounds = {
    'changepoint_prior_scale': (0.001, 0.5),
    'seasonality_prior_scale': (0.1, 20),
    'holidays_prior_scale': (0.1, 20)
}

optimizer = BayesianOptimization(
    f = Prophet_cv,
    pbounds = pbounds,
    random_state = 42
)

optimizer.maximize(init_points = 5, n_iter = 25)
best_params = optimizer.max['params']
print('Best params is:', best_params)

In [ ]:
best_param = optimizer.max['params']
best_model = Prophet(
    changepoint_prior_scale = best_param['changepoint_prior_scale'],
    seasonality_prior_scale = best_param['seasonality_prior_scale'],
    holidays_prior_scale = best_param['holidays_prior_scale']
)
best_model.fit(df1)

future = best_model.make_future_dataframe(periods=len(test_df))
forecast = best_model.predict(future)

forecast

In [ ]:
comparison_df = pd.DataFrame({
    'ds': test_df['ds'].values,
    'y_true': test_df['y'].values,
    'y_pred': forecast['yhat'].iloc[-len(test_df):].values
})

comparison_df


In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(comparison_df['y_true'], comparison_df['y_pred'])
mae

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
plt.plot(comparison_df['ds'], comparison_df['y_true'], label='Real', linewidth=2)
plt.plot(comparison_df['ds'], comparison_df['y_pred'], label='Predicted', linewidth=2)
plt.legend()
plt.xticks(rotation=45)
plt.show()


In [ ]:
!pip install neuralprophet


In [ ]:
from neuralprophet import NeuralProphet

print("NeuralProphet imported successfully!")


In [ ]:
!pip install neuralprophet


In [ ]:
from neuralprophet import NeuralProphet

print("NeuralProphet imported successfully!")

In [ ]:
conda create -n prophet python=3.10
conda activate prophet
pip install neuralprophet


In [ ]:
!pip install neuralforecast


In [ ]:
df_fc = df1[['ds', 'y']].copy()
df_fc['unique_id'] = 'series_1'


In [ ]:
H = len(test_df)

train_fc = df_fc.iloc[:-H]
test_fc  = df_fc.iloc[-H:]


In [ ]:
df_fc.dtypes


In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS

model = NHITS(
    h=H,
    input_size=36,
    max_steps=300
)

fcst = NeuralForecast(
    models=[model],
    freq='M'
)

fcst.fit(train_fc)
forecast = fcst.predict()






In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14,6))

plt.plot(test_fc['ds'], test_fc['y'], label='Real', linewidth=2)
plt.plot(test_fc['ds'], forecast['NHITS'], label='NHITS Forecast', linewidth=2)

plt.title("Real vs NHITS Forecast")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()
